<a href="https://colab.research.google.com/github/ankojubhanuprakash/grain-segmentation/blob/main/Prototype_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import os
import cv2
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.cluster import KMeans
#from wmi import WMI
import logging
from PIL import Image
import matplotlib.pyplot as plt
#from playsound import playsound
#from google.colab import drive
# drive.mount('/content/drive')
########
import threading
import shutil
import queue
import warnings
warnings.filterwarnings('ignore')
import time


In [58]:
class GrainSegment:
    def __init__(self) -> None:
        #initlilise default parameters
        self.contours_add = []
        self.contours_normal = []
        self.contours_abnormal = []
        self.contours_remove = []
        self.contours_remove_normal_copy = []
        self.contours_remove_abnormal = []
        self.contours_remove_abnormal_copy= []
        self.contours_remove_normal = []
        self.line_manual = []

        self.list_balance_State = []
        self.list_light_State = []

        self.num_process = 0
        self.num_contours = 0
        self.meanArea = 0
        self.meanPerimeter = 0
        self.meanLength = 0
        self.meanWidth = 0
        self.shortEdge = 0

        self.trigger_zoom = 0
        self.mouse_mv_x = ""
        self.mouse_mv_y = ""
        self.labelx = 0
        self.labely = 0

        self.ratio = arearatio
        self.area_calibration_true = 0
        self.mouseButton = 0

        self.mouseButton = 0


        self.kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        try:
            with open("model.txt", "r") as f:
                self.grainModel = int(f.readline().strip("\n"))
        except:
            self.grainModel = 0

    def openimage(self,imagename):
        print('openimage')
        self.imageName = imagename
        self.image_open =  cv2.imread(self.imageName, )
        self.image_color = self.image_open.copy()

        self.image_open_display = self.image_open.copy()
        #self.image_open_display = cv2.resize(self.image_open_display, (self.image_width, self.image_height))

    def findThreshold(self,image):

      hist = cv2.calcHist([image], [0], None, [256], [0, 256])
      hist_flat= hist.flatten()
      peakindex = signal.find_peaks_cwt(hist_flat,(15,15) )
      if len(peakindex) == 2:
        return int(np.mean(peakindex))

      else:
        return 110
    def watershed(self, binary, factor_distance):
      binary_BGR = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
      # DIST_L2 =
      dist_watershed = cv2.distanceTransform(binary, cv2.DIST_L2, 5)

      ret, foreground = cv2.threshold(dist_watershed, dist_watershed.max() * factor_distance, 255, cv2.THRESH_BINARY)
      foreground = np.uint8(foreground)
      kernel_foreground = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
      foreground = cv2.morphologyEx(foreground, cv2.MORPH_OPEN, kernel_foreground)
      unknown = cv2.subtract(binary, foreground)
      _, labels = cv2.connectedComponents(foreground)
      labels = labels + 1
      labels[unknown == 255] = 0
      labels = cv2.watershed(binary_BGR, labels)
      binary[labels == -1] = 0
      return binary

    def findratioarea(self, binary):
        # Find adhesion areas based on area ratio
        # print('findratioarea')
        contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        list_grainArea = []
        list_convexArea = []
        list_ratioArea = []

        list_grainArea = [ cv2.contourArea(i) for i in contours]
        list_convexArea = [ cv2.contourArea(cv2.convexHull(i)) for i in contours]
        list_ratioArea = [grain_area/convex_area if convex_area != 0 else 0  for grain_area,convex_area
                          in zip(list_grainArea,list_convexArea)]


        return contours, hierarchy, list_ratioArea, list_grainArea

    def findshapefactor(self, binary):
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        list_grainarea = [ cv2.contourArea(i ) for i in contours ]
        list_grainArea = [4 * 3.141592653 * i for i in list_grainarea ]
        list_grainPerimeter = [ cv2.arcLength(i, True) ** 2 for  i in contours ] ## FLAG

        list_shapeFactor = [grain_area/grain_perimeter if grain_perimeter != 0 else 0  for grain_area,grain_perimeter
                          in zip(list_grainArea,list_grainPerimeter)]



        list_longEdge = []
        list_shortEdge = []

        for i in range(len(contours)):
            rect = cv2.minAreaRect(contours[i])
            if rect[1][0] >= rect [1][1]:
                list_longEdge.append(rect[1][0])
                list_shortEdge.append(rect[1][1])

            else:
                list_longEdge.append(rect[1][1])
                list_shortEdge.append(rect[1][0])

        longEdge = np.mean(list_longEdge)

        if list_shortEdge == []:
            shortEdge = 40
        else:
            shortEdge = np.mean(list_shortEdge)


        return contours, list_shapeFactor, list_grainarea, longEdge, shortEdge

    def shapeFactorAndArea(self, inplist, way):
        # print('shapeFactorAndArea has kmeans')
        inplist.sort()
        for i in range(len(inplist)):
            inplist[i] = round(inplist[i], 2)

        if len(inplist) >= 3:
            kmeans = KMeans(n_clusters = 3)
        elif len(inplist) == 2:
            kmeans = KMeans(n_clusters= 2 )
        else:
            kmeans = KMeans(n_clusters= 1 )

        list_reshape = np.array(inplist).reshape(-1, 1)
        kmeans.fit(list_reshape)
        list_single = []

        if way == 0:
            for i in range(len(inplist)):
                if kmeans.labels_[i] == kmeans.labels_[-1]:
                    list_single.append(list[i])
        else:
            for i in range(len(inplist)):
                if kmeans.labels_[i] == kmeans.labels_[0]:
                    list_single.append(inplist[i])

        return np.mean(list_single)
    def processing(self):
      channel_B=cv2.imread(self.imageName, cv2.IMREAD_GRAYSCALE)
      thre = self.findThreshold(channel_B)
      _, binary = cv2.threshold(channel_B, thre, 255, cv2.THRESH_BINARY)
      ## Simple-touching grain segmentation
      binary_seg_shape = binary.copy()
      binary_seg_shape = self.watershed(binary_seg_shape, 0.3)
      # plt.imshow(binary_seg_shape)
      # plt.show()
      binary_seg_shape = cv2.morphologyEx(binary_seg_shape, cv2.MORPH_OPEN, self.kernel, iterations=2)
      binary_seg_shape = cv2.erode(binary_seg_shape, self.kernel)
      # plt.imshow(binary_seg_shape)


      contours_shape, _, _, list_grainArea_shape = self.findratioarea(binary_seg_shape )
      # print
      if len(list_grainArea_shape) != 0:
        for i in range(len(list_grainArea_shape)):
            if list_grainArea_shape[i] <= 0.1 * np.mean(list_grainArea_shape):
                cv2.drawContours(binary_seg_shape, [contours_shape[i]], 0, 0, -1)

      _, list_shapeFactor_checkshape, list_grainArea, self.longEdge, self.shortEdge = self.findshapefactor(binary_seg_shape)
      self.shapeFactor_checkshape = self.shapeFactorAndArea(list_shapeFactor_checkshape,0)
      # print(f'first call')
      self.area_checkshape = self.shapeFactorAndArea(list_grainArea,1)

In [59]:
arearatio = 6.732629815077101e-05
a=GrainSegment()

a.openimage('21.Bmp')
a.processing()

openimage


TypeError: unsupported operand type(s) for +: 'types.GenericAlias' and 'types.GenericAlias'

In [ ]:
# i = time.time()
%timeit a.processing()
# print(time.time()-i)

In [6]:
!ls

21.Bmp	sample_data


In [21]:
a= [0,1,2,3,4]
[x/y if y != 0 else 0  for x,y in zip(a,a)]

[0, 1.0, 1.0, 1.0, 1.0]

In [28]:
time.time()

1710672770.8152068

In [44]:

def fna():
  x=1000
  a=[]
  b=[]
  c= []
  d= []
  for i in range(x):
    a.append(i)
    b.append(i)
    c.append(i)
    d.append(i)
%timeit fna()

123 µs ± 2.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [43]:

def fnb():
  x=1000
  a=[]
  b=[]
  c= []
  d= []
  a=[]
  a= [i for i in range(x)]
  b= [i for i in range(x)]
  c= [i for i in range(x)]
  d= [i for i in range(x)]

%timeit fnb()

98.4 µs ± 601 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
